In [1]:
# Imports and config for the notebook

## Notebook config
import sys
if '../' not in sys.path:
    sys.path.append("../")
%load_ext dotenv
%reload_ext dotenv
%dotenv

import collections
import os
import csv

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import graphistry

from datasources.neo4j import gds
from queries import utils

/home/ec2-user/miniconda3/envs/rnalab/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Configs

# graphistry.register(
#     api=3,
#     username=os.getenv('GRAPHISTRY_USERNAME'),
#     password=os.getenv('GRAPHISTRY_PASSWORD'),
# )

RANDOM_SEED = 42
EMBEDDINGS_DIR = '/mnt/embeddings/'

print(gds)

TODO:


- Create Tissue projection (random generate initial embeddings)
- Create Taxon projection (use rank as initial feature?)

- Create heterogenous projection from dataset?
- run memory estimates for hashgnn and fastRP using hetero projection and 


- HashGNN_homogenous, HashGNN_heterogenous
- FastRP_homogenous, FastRP_heterogenous

In [13]:
TAXON_PROJECTION_NAME = 'taxon-graph'
TISSUE_PROJECTION_NAME = 'tissue-graph'
HOMOGENOUS_PROJECTION_NAME = 'homogenous-graph'
HETERO_PROJECTION_NAME = 'hetero-graph'


In [14]:

def get_homogenous_projection():
    projection = gds.graph.project(
        graph_name=HOMOGENOUS_PROJECTION_NAME,
        node_spec=[
            'Taxon',
            'Tissue',
            'SOTU',
        ],
        relationship_spec={
            'HAS_PARENT': {'orientation': 'UNDIRECTED'},
            'SEQUENCE_ALIGNMENT': {'orientation': 'UNDIRECTED'},
        },
    )
    return projection


# TODO: Use dataset projection
def get_heterogenous_projection():
    projection = gds.graph.project(
        graph_name=HETERO_PROJECTION_NAME,
        node_spec=[
            'Taxon',
            'Tissue',
            'SOTU',
        ],
        relationship_spec={
            'HAS_PARENT': {'orientation': 'UNDIRECTED'},
            'SEQUENCE_ALIGNMENT': {'orientation': 'UNDIRECTED'},
        },
    )
    return projection


def get_taxon_projection():
    projection = gds.graph.project(
        graph_name=TAXON_PROJECTION_NAME,
        node_spec=[
            'Taxon',
        ],
        relationship_spec={
            'HAS_PARENT': {'orientation': 'UNDIRECTED'},
        },
    )
    return projection

def get_tissue_projection():
    projection = gds.graph.project(
        graph_name=TISSUE_PROJECTION_NAME,
        node_spec=['Tissue'],
        relationship_spec={
            'HAS_PARENT': {'orientation': 'UNDIRECTED'},
        },
    )
    return projection


In [15]:
projection = get_homogenous_projection()

In [18]:
# https://neo4j.com/docs/graph-data-science/current/machine-learning/node-embeddings/hashgnn/#algorithms-embeddings-hashgnn-syntax
# https://github.com/neo4j/graph-data-science-client/blob/main/examples/heterogeneous-node-classification-with-hashgnn.ipynb#L18

# one may try to set embeddingDensity to 128, 256, 512, or roughly 25%-50% of the embedding dimension, i.e. the number of binary features.

gds.hashgnn.stream.estimate(
    G=gds.graph.get(HOMOGENOUS_PROJECTION_NAME),
    nodeLabels=['Taxon', 'Tissue', 'SOTU'],
    relationshipTypes=['HAS_PARENT', 'SEQUENCE_ALIGNMENT'],
    randomSeed=RANDOM_SEED,
    generateFeatures={
        'dimension': 512, # dimension of the embedding vector
        'densityLevel': 2, # number of initial values equalling 1
    },
    iterations=10, # maximum number of hops
    embeddingDensity=256,
    neighborInfluence=1.0,
)

requiredMemory                                [1338 MiB ... 12865 MiB]
treeView             Memory Estimation: [1338 MiB ... 12865 MiB]\n|...
mapView              {'memoryUsage': '[1338 MiB ... 12865 MiB]', 'n...
bytesMin                                                    1403659432
bytesMax                                                   13490131432
nodeCount                                                      3021618
relationshipCount                                             57702540
heapPercentageMin                                                  0.1
heapPercentageMax                                                  0.6
Name: 0, dtype: object

In [20]:
filename = 'HashGNN_homogenous.csv'
df = gds.hashgnn.stream(
    G=gds.graph.get(HOMOGENOUS_PROJECTION_NAME),
    nodeLabels=['Taxon', 'Tissue', 'SOTU'],
    relationshipTypes=['HAS_PARENT', 'SEQUENCE_ALIGNMENT'],
    randomSeed=RANDOM_SEED,
    generateFeatures={
        'dimension': 512, # dimension of the embedding vector
        'densityLevel': 2, # number of initial values equalling 1
    },
    iterations=10, # maximum number of hops
    embeddingDensity=256,
    neighborInfluence=1.0,
)

df.to_csv(EMBEDDING_DIR + filename, index=False)

HashGNN:  48%|████▊     | 47.58/100 [1:53:11<2:18:41, 158.75s/%]

In [ ]:
gds.fastrp.stream.estimate(
    G=gds.graph.get(HOMOGENOUS_PROJECTION_NAME),
    nodeLabels=['Taxon', 'Tissue', 'SOTU'],
    relationshipTypes=['HAS_PARENT', 'SEQUENCE_ALIGNMENT'],
    randomSeed=RANDOM_SEED,
    embeddingDimension=256,
    relationshipWeightProperty="weight",
)

In [ ]:
filename = 'FastRP_homogenous.csv'
df = gds.fastrp.stream(
    G=gds.graph.get(HOMOGENOUS_PROJECTION_NAME),
    nodeLabels=['Taxon', 'Tissue', 'SOTU'],
    relationshipTypes=['HAS_PARENT', 'SEQUENCE_ALIGNMENT'],
    randomSeed=RANDOM_SEED,
    embeddingDimension=256,
    relationshipWeightProperty='weight',
)
df.to_csv(EMBEDDING_DIR + filename, index=False)

In [2]:
from queries import feature_queries
from config.base import (
    DIR_CFG,
    MODEL_CFG,
    DATASET_CFG,
)

dir_name = '/mnt/graphdata/query_cache/neo4j/'


dataset_cfg = DATASET_CFG
nodes = feature_queries.get_all_node_features(
    dir_name=dir_name,
    dataset_cfg=dataset_cfg,
)

relationships = feature_queries.get_all_relationship_features(
    dir_name=dir_name,
    dataset_cfg=dataset_cfg,
)

undirected_relationship_types = list(
    map(
        (lambda cfg: cfg['TYPES'][0]),
        dataset_cfg['REL_TYPES']
    )
)

Reading local file:  /mnt/graphdata/query_cache/neo4j/sotu_nodes.csv
Reading local file:  /mnt/graphdata/query_cache/neo4j/taxon_nodes.csv
Reading local file:  /mnt/graphdata/query_cache/neo4j/tissue_nodes.csv
Reading local file:  /mnt/graphdata/query_cache/neo4j/sotu_has_host_stat_edges.csv
Reading local file:  /mnt/graphdata/query_cache/neo4j/taxon_has_parent_edges.csv
Reading local file:  /mnt/graphdata/query_cache/neo4j/sotu_sequence_alignment_edges.csv
Reading local file:  /mnt/graphdata/query_cache/neo4j/sotu_has_inferred_taxon_edges.csv
Reading local file:  /mnt/graphdata/query_cache/neo4j/sotu_has_tissue_metadata_edges.csv


In [6]:
print(nodes.shape)
print(relationships.shape)

model_cfg = MODEL_CFG
sampling_ratio = 1

graph_name = \
        f"{model_cfg['PROJECTION_NAME']}_{sampling_ratio}"

print(graph_name)

(3021618, 3)
(24096965, 4)
incl-best-1_1


In [9]:
gds.alpha.graph.construct(
    graph_name=graph_name,
    nodes=nodes,
    relationships=relationships,
    concurrency=40,
    undirected_relationship_types=undirected_relationship_types,
)

KeyboardInterrupt: 

In [13]:
result = gds.graph.project.estimate(
    '*',
    '*',
)

print(result)
print(result['requiredMemory'])

KeyboardInterrupt: 